In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
import tensorflow as tf
import imageio
import cv2
import mediapipe as mp

### 학습데이터 준비

In [2]:
dataset_path = os.listdir('dataset/train')
label_types = os.listdir('dataset/train')
# 훈련 데이터셋을 위한 비어있는 리스트 초기화
rooms = []
# dataset_path에 저장된 각 항목(방 유형)에 대해 반복
for item in dataset_path:
    # 'dataset/train' 폴더 내 각 방 유형별로 모든 파일 이름을 가져옴
    all_rooms = os.listdir('dataset/train'+'/'+item)    
    # 가져온 파일 이름을 rooms 리스트에 추가
    for room in all_rooms:
        rooms.append((item, str('dataset/train'+'/'+item)+'/'+room))
# rooms 리스트를 사용하여 데이터프레임 생성
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name']).loc[:,['video_name','tag']]
df = train_df.loc[:,['video_name','tag']]
# 생성된 데이터프레임을 CSV 파일로 저장
df.to_csv('train.csv', encoding='utf-8-sig')
df

,video_name,tag
0,dataset/train/가볍다/1.mp4,가볍다
1,dataset/train/가져오다/2.mp4,가져오다
2,dataset/train/가짜/3.mp4,가짜
3,dataset/train/가치/4.mp4,가치
4,dataset/train/보관/5.mp4,보관
5,dataset/train/보내다/6.mp4,보내다
6,dataset/train/보다/7.mp4,보다
7,dataset/train/안경/8.mp4,안경
8,dataset/train/알다/9.mp4,알다
9,dataset/train/월요일/10.mp4,월요일


### 테스트 데이터 준비

In [3]:
dataset_path = os.listdir('dataset/test')
room_types = os.listdir('dataset/test')

rooms = []
# dataset_path에 저장된 각 항목(방 유형)에 대해 반복
for item in dataset_path:
    # 'dataset/test' 폴더 내 각 방 유형별로 모든 파일 이름을 가져옴
    all_rooms = os.listdir('dataset/test'+'/'+item)
    # 가져온 파일 이름을 rooms 리스트에 추가
    for room in all_rooms:
        rooms.append((item, str('dataset/test'+'/'+item)+'/'+room))

# rooms 리스트를 사용하여 데이터프레임 생성
train_df = pd.DataFrame(data=rooms, columns=['tag','video_name'])
df = train_df.loc[:,['video_name','tag']]
# 생성된 데이터프레임을 CSV 파일로 저장
df.to_csv('test.csv', encoding='utf-8-sig')
df

,video_name,tag
0,dataset/test/가볍다/1.mp4,가볍다
1,dataset/test/가져오다/2.mp4,가져오다
2,dataset/test/가짜/3.mp4,가짜
3,dataset/test/가치/4.mp4,가치
4,dataset/test/보관/5.mp4,보관
5,dataset/test/보내다/6.mp4,보내다
6,dataset/test/보다/7.mp4,보다
7,dataset/test/안경/8.mp4,안경
8,dataset/test/알다/9.mp4,알다
9,dataset/test/월요일/10.mp4,월요일


In [4]:
# !pip install git+https://github.com/tensorflow/docs

In [5]:
# GPU 설정
gpus = tf.config.list_physical_devices('GPU')
# GPU가 사용 가능한 경우
if gpus:
    try:
        # 첫 번째 GPU에 대해 메모리 제한 설정
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], 
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)]
        )
    except RuntimeError as e:
        print(e)

### Data preparation

In [6]:
# 훈련 데이터셋과 테스트 데이터셋을 각각 CSV 파일에서 로드
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 훈련 및 테스트 데이터셋 크기 출력
print(f"Total video for training: {len(train_df)}")
print(f"Total video for testing: {len(test_df)}")

# 훈련 데이터셋의 샘플 5개 출력
train_df.sample(5)

Total video for training: 10
Total video for testing: 10


,Unnamed: 0,video_name,tag
9,9,dataset/train/월요일/10.mp4,월요일
4,4,dataset/train/보관/5.mp4,보관
8,8,dataset/train/알다/9.mp4,알다
7,7,dataset/train/안경/8.mp4,안경
6,6,dataset/train/보다/7.mp4,보다


### Feed the video to a network:

In [7]:
IMG_SIZE = 224

mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, smooth_landmarks=True)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

In [8]:
# 주어진 이미지에서 중앙에 맞춰 정사각형으로 잘나내는 함수
def crop_center_square(frame):
    # 이미지의 높이(y)와 너비(x)를 가져옴
    y, x = frame.shape[0:2]
    # 이미지의 높이와 너비 중 더 작은 값을 선택하여 정사각형의 크기를 결정
    min_dim = min(y, x)
    # 정사각형을 이미지 중앙에 위치시키기 위해 시작점의 x좌표와 y좌표를 계산
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    # 계산된 시작점과 정사각형의 크기를 이용하여 이미지의 중앙 부분을 잘라냅니다.
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


# 비디오 파일을 로드하고, 각 프레임을 처리하여 배열로 반환하는 함수
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    # OpenCV를 사용하여 비디오 파일 열기
    cap = cv2.VideoCapture(path)
    frames = []
    skeletons = []  # 스켈레톤 데이터
    hand_landmarks = []  # 손 데이터
    face_landmarks = []  # 얼굴 데이터
    try:
        while True:
            # 비디오에서 프레임을 하나씩 읽기
            ret, frame = cap.read()
            # 읽을 프레임이 없으면 반복문을 종료
            if not ret:
                break
            # 읽은 프레임에서 중앙의 정사각형 부분을 잘라냄
            frame = crop_center_square(frame)
            # 프레임의 크기를 지정된 크기로 조절
            frame = cv2.resize(frame, resize)            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Mediapipe를 사용하여 스켈레톤 추출
            hands_results = hands.process(frame_rgb)
            pose_results = pose.process(frame_rgb)
            face_results = face_detection.process(frame_rgb)
           
            if pose_results.pose_landmarks:
                skeletons.append(pose_results.pose_landmarks.landmark)
            if hands_results.multi_hand_landmarks:
                hand_landmarks.append(hands_results.multi_hand_landmarks)
            if face_results.detections:
                face_landmarks.append(face_results.detections)
            
            # OpenCV는 BGR 색상 순서를 사용하므로, 이를 RGB 순서로 변경
            frame = frame[:, :, [2, 1, 0]]
            # 처리된 프레임을 프레임 리스트에 추가
            frames.append(frame)
            # max_frames가 지정된 경우, 지정된 수의 프레임만큼만 처리
            if len(frames) == max_frames:
                break
    finally:
        # 비디오 파일을 닫기
        cap.release()
        pose.close
        hands.close
        face_detection.close
    # 처리된 모든 프레임을 numpy 배열로 변환하여 반환
    return np.array(frames), skeletons, hand_landmarks, face_landmarks

### 특징 추출

In [9]:
def build_feature_extractor():
    # 이미지 특징 추출을 위한 InceptionV3 모델
    base_model = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input
    image_input = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed_image = preprocess_input(image_input)
    image_features = base_model(preprocessed_image)

    # Mediapipe 데이터를 위한 입력 레이어 및 처리 레이어
    # 예시로, Mediapipe 데이터의 차원을 상정하여 입력 레이어를 정의
    mediapipe_input = keras.Input((1599+63,))
    mediapipe_features = keras.layers.Dense(128, activation="relu")(mediapipe_input)

    # 이미지 특징과 Mediapipe 데이터의 결합
    combined_features = keras.layers.concatenate([image_features, mediapipe_features])

    # 최종 모델
    outputs = keras.layers.Dense(10, activation="softmax")(combined_features)
    return keras.Model(inputs=[image_input, mediapipe_input], outputs=outputs, name="feature_extractor")

### Label Encoding
- StringLookup layer encode the class labels as integers

In [10]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[...,None]).numpy()
labels

['가볍다', '가져오다', '가짜', '가치', '보관', '보내다', '보다', '안경', '알다', '월요일']


array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]], dtype=int64)

In [11]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048
SKELETON_FEATURES = 33*4
HAND_FEATURES = 21*3*2
FACE_FEATURES = 468*3

In [12]:
def prepare_all_video(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # 수정: Mediapipe 데이터를 저장할 배열 초기화
    frame_skeletons = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, SKELETON_FEATURES), dtype="float32")
    frame_hands = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, HAND_FEATURES), dtype="float32")
    frame_faces = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, FACE_FEATURES), dtype="float32")

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")
    
    for idx, path in enumerate(video_paths):
        frames, skeletons, hands, faces = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        video_length = min(MAX_SEQ_LENGTH, frames.shape[1])

        for i in range(video_length):
            frame_features[idx, i, :] = feature_extractor.predict(frames[0, i, :])

            # Mediapipe 데이터의 전처리 및 저장
            frame_skeletons[idx, i, :] = preprocess_skeleton_data(skeletons[i])
            frame_hands[idx, i, :] = preprocess_hand_data(hands[i])
            frame_faces[idx, i, :] = preprocess_face_data(faces[i])

            frame_masks[idx, i] = 1

    # 수정: 반환 값에 Mediapipe 데이터 포함
    return (frame_features, frame_skeletons, frame_hands, frame_faces, frame_masks), labels


train_data, train_labels = prepare_all_video(train_df, "train")
test_data, test_labels = prepare_all_video(test_df,"test")
train_labels = np.squeeze(train_labels)
test_labels = np.squeeze(test_labels)

print(f"Frame feature in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

print(f"train_labels in train set:{train_labels.shape}")
print(f"test_labels in train set:{test_labels.shape}")

Frame feature in train set: (10, 20, 2048)
Frame masks in train set: (10, 20, 132)
train_labels in train set:(10,)
test_labels in train set:(10,)


### The sequence model

In [13]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()
    
    # 기존 이미지 특징에 대한 입력
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    # Mediapipe 데이터에 대한 추가 입력 레이어
    skeleton_input = keras.Input((MAX_SEQ_LENGTH, SKELETON_FEATURES))
    hand_input = keras.Input((MAX_SEQ_LENGTH, HAND_FEATURES))
    face_input = keras.Input((MAX_SEQ_LENGTH, FACE_FEATURES))
    
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")
    
    # 이미지 특징 처리를 위한 GRU 레이어
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8, return_sequences=True)(x)
    x = keras.layers.GlobalAveragePooling1D()(x)  # 형태 변경
    
    # Mediapipe 데이터를 처리하는 추가 네트워크 레이어 (예시)
    # 여기서는 간단히 Dense 레이어를 사용했지만, 필요에 따라 다른 구조를 사용할 수 있습니다.
    y_skeleton = keras.layers.GlobalAveragePooling1D()(skeleton_input)
    y_hand = keras.layers.GlobalAveragePooling1D()(hand_input)
    y_face = keras.layers.GlobalAveragePooling1D()(face_input)

    # 모든 특징을 결합
    combined = keras.layers.concatenate([x, y_skeleton, y_hand, y_face])

    # 결합된 특징에 대한 추가 처리
    z = keras.layers.Dense(16, activation="relu")(combined)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(z)
    
    rnn_model = keras.Model([frame_features_input, skeleton_input, hand_input, face_input, mask_input], output)
    rnn_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return rnn_model

EPOCHS = 30

def run_experiment():
    filepath = "./tmp/video_classifier.h5"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1], train_data[2], train_data[3], train_data[4]],  # 수정된 입력 데이터
        train_labels,
        batch_size=1,
        validation_split=0.2,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate(
        [test_data[0], test_data[1], test_data[2], test_data[3], test_data[4]],  # 수정된 입력 데이터
        test_labels
    )
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

_, sequence_model = run_experiment()

Epoch 1/30
7/8 [=========================>....] - ETA: 0s - loss: nan - accuracy: 0.1429     
Epoch 1: val_loss did not improve from inf
8/8 [==============================] - 9s 335ms/step - loss: nan - accuracy: 0.1250 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/30
8/8 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1250    
Epoch 2: val_loss did not improve from inf
8/8 [==============================] - 0s 13ms/step - loss: nan - accuracy: 0.1250 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/30
8/8 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1250    
Epoch 3: val_loss did not improve from inf
8/8 [==============================] - 0s 13ms/step - loss: nan - accuracy: 0.1250 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/30
7/8 [=========================>....] - ETA: 0s - loss: nan - accuracy: 0.1429    
Epoch 4: val_loss did not improve from inf
8/8 [==============================] - 0s 13ms/step - loss: nan - accuracy: 

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = './tmp/video_classifier.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### Inference

In [ ]:
def prepare_single_video(frames, skeletons, hands, faces):
    frames = frames[None,...]
    frame_mask = np.zeros((1, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros((1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # Mediapipe 데이터를 위한 배열 초기화
    frame_skeletons = np.zeros((1, MAX_SEQ_LENGTH, SKELETON_FEATURES), dtype="float32")
    frame_hands = np.zeros((1, MAX_SEQ_LENGTH, HAND_FEATURES), dtype="float32")
    frame_faces = np.zeros((1, MAX_SEQ_LENGTH, FACE_FEATURES), dtype="float32")

    video_length = min(MAX_SEQ_LENGTH, frames.shape[1])

    for j in range(video_length):
        frame_features[0, j, :] = feature_extractor.predict(frames[0, j, :])

        # Mediapipe 데이터 처리 및 저장
        frame_skeletons[0, j, :] = preprocess_skeleton_data(skeletons[j])
        frame_hands[0, j, :] = preprocess_hand_data(hands[j])
        frame_faces[0, j, :] = preprocess_face_data(faces[j])

        frame_mask[0, j] = 1

    return frame_features, frame_skeletons, frame_hands, frame_faces, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()
    
    frames, skeletons, hands, faces = load_video(os.path.join("test", path))
    frame_features, frame_skeletons, frame_hands, frame_faces, frame_mask = prepare_single_video(frames, skeletons, hands, faces)

    probabilities = sequence_model.predict([frame_features, frame_skeletons, frame_hands, frame_faces, frame_mask])[0]
    
    for i in np.argsort(probabilities)[::-1]:
        print(f"{class_vocab[i]} : {probabilities[i] * 100:5.2f}%")
    return frames

    
test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path : {test_video}")

test_frames = sequence_prediction(test_video)